In [2]:
import numpy as np

In [10]:
Pi = 3.14159265358979323846
COLDLAT = 0
HOTLAT = 1
PLUS = 1
MINUS = -1
EVEN = 0x02
ODD = 0x01
EVENANDODD = 0x03
LSIZE = 16  # change LSIZE to 16/32 accordingly
DELTAMAX = 50.0
# acl parameters for switch = 0
T_cut = 5
D_cut = 5
MAXT_cut = 25
NOT_cut = 5
# acl parameters for switch = 1
# T_cut = 15
# D_cut = 5
# MAXT_cut = 15
# NOT_cut = 1

# With the "EVENFIRST" option the even sites are listed contiguously
# in the first part of lattice[], and the odd sites in the last part.
# In this version there must be an equal number of even and odd sites
# in each plane - in other words one of the two shortest dimensions must
# be even.

# EVENFIRST = True

nx = 0  # lattice dimensions
nt = 0
volume = 0  # nx * nt
nf = 0  # # of flavors

mdstep = 0  # # of molecular dynamics step
cgiter1 = 0  # # of conjugate grad. iterations
cgiter2 = 0
iseed = 0  # random number seed
g = 0.0  # Gross_Neveu coupling
step = 0.0  # leap frog step size
residue1 = 0.0  # residue for cgiter1 and 2
residue2 = 0.0
mid = 0  # middle of the lattice
no_even_sites = 0  # # of even sites
no_odd_sites = 0  # # of odd sites
no_garbage = 0  # # of garbage loops
bin_length = 0  # length of each bin in garbage loops
no_bin = 0  # # of bins
meas_loop = 0  # # of measurement loops after garbage
meas_length = 0  # length after which each meas. is made
prop_length = 0  # length after which Acl. calculation on propagator is made

no_meas = 0  # no_meas = meas_loop/meas_length
seg_length = 0  # length of each segments
no_a_seg = 0  # no. of segments for autocoreln. meas.
no_prop_seg = 0  # no. of segments for Prop-Acl calcu
hmc_it = 0  # # of HMC sweeps, counting only accepted ones
counter = 0  # counter for HMC loop
sw_flag = 0  # flag to switch action between garbage and autocorln. loops and measurement loops. sw_flag = 0 => garbage & autocorln. loop; sw_flag = 1 => measurement loop;

class site:
	def __init__(self):
		self.x = 0  # co ordinates of this site
		self.t = 0
		self.sign = 0  # phase for staggered fermions
		self.parity = 0  # even or odd site
		self.sigma = 0.0  # real scalar field
		self.phi = 0.0  # temporary space for sigma
		self.mom = 0.0  # canonical momenta of sigma
		self.chi = np.zeros(4, dtype=np.float64)  # pseudo fermionic field, source vector
		self.eta = np.zeros(4, dtype=np.float64)  # Gaussian noise, from which chi is derived, also used for dest vector etc
		self.p = 0.0  # pseudofermionic vector for CG
		self.r = 0.0  # pseudofermionic vector for CG, used for residue
		self.mp = 0.0  # pseudofermionic vector for CG
		self.mmp = 0.0  # pseudofermionic vector for CG
  
lattice = np.empty(LSIZE, dtype=site)  # lattice structure

store = np.zeros(LSIZE, dtype=np.float64)  # storage space for sigma configuration while the code is running.
conf = np.zeros(LSIZE, dtype=np.float64)  # storage space for sigma config. generated in the last accepted loop.
con = np.zeros(LSIZE, dtype=np.float64)  # storage space for sigma config. generated in the last but one loop.
garbage = np.zeros(LSIZE, dtype=np.float64)  # storage for <sigma> during garbage loops
ac_store = np.zeros(LSIZE, dtype=np.float64)  # storage for <sigma> during autocrln. loops
ac_prop = np.zeros(LSIZE, dtype=np.float64)  # storage for propagator during autocrln. calculation loops
bin_av = np.zeros(LSIZE, dtype=np.float64)  # storage space for bin averages
psi = np.zeros(LSIZE, dtype=np.float64)  # storage space for <psi_bar-psi>
psi_acl = np.zeros(LSIZE, dtype=np.float64)  # storage space for <psi_bar_psi> for Acl. calculation for each confign.
G_prop = np.zeros(LSIZE, dtype=np.float64)  # storage for propagator
G_store = [np.zeros(LSIZE, dtype=np.float64) for _ in range(LSIZE)]  # storage for propagator
G_temp = [np.zeros(LSIZE, dtype=np.float64) for _ in range(LSIZE)]  # storage for propagator for each confign.
prop = np.zeros(LSIZE, dtype=np.float64)  # working storage for propagator
tprop = np.zeros(LSIZE, dtype=np.float64)  # working storage for propagator
T_int = np.zeros((NOT_cut, MAXT_cut), dtype=np.float64)  # storage for t_int for various segments
T_int_prop = np.zeros((NOT_cut, MAXT_cut, LSIZE), dtype=np.float64)  # storage for t_int for Prop-Acl

gen_pt = None  # vectors gen_pt[lattice_site][pointer_number] for gather routine etc
# N_POINTERS = 1
# gen_pt = [None] * N_POINTERS

# Autocorel.c

In [1]:
def autocorel(sigma_av, lb, a_index):
    
    global T_cut, MAX_Tcut, seg_length, ac_store, NOT_cut, rho, T_int, D_cut
    
    c0 = 0.0; N_t = 0; tcut = T_cut
    
    #Calculating the unnormalized autocorrelation functions ct and c0
    #and normalized autocorrelation function rho.
    
    for j in range(lb, lb+seg_length):
        c0 += ((ac_store[j] - sigma_av) * (ac_store[j] - sigma_av))/seg_length
    
    for u in range(0, NOT_cut):
        for t in range(1, tcut+1):
            N_t = seg_length - t
            ct = 0.0
            for j in range(lb, lb+N_t):
                ct += ((ac_store[j] - sigma_av) * (ac_store[j+t] - sigma_av))/N_t
                rho[t-1] += ct/c0
        
        for t in range(0, tcut): T_int[u][t][a_index] = 0.5
        
        #Calculation of tau_int, the integrated autocorrelation time
        for t in range(0, tcut):
            for k in range(0, t+1): T_int[u][t][a_index] += rho[k]
        
        tcut += D_cut

# Average_sigma.c

In [ ]:
def average_sigma(sites):
    
    '''This program calculates the expectation value of sigma-field
    by summing all the sigmas' in the sites and dividing by the 
    volume. This <sigma> is stored in store[] and it returns a
    double <sigma>. '''
   
    t_sigma = sum(site.sigma for site in sites)
    av_sigma = t_sigma / len(sites)
    return av_sigma

# matd2d.c

In [ ]:
def matd2d(src, dest, isign, parity):
   """
   This function multiplies the fermion matrix or its adjoint with a pseudovector.
   It accepts a "double" and returns a "double".
   """
   def gather(field, index, parity, dest):
      pass  # Placeholder for gather function

   gather(src, XUP, EVENANDODD, gen_pt)
   for s in sites:
      dest[s] = s.sign * 0.5 * gen_pt[s]

   gather(src, XDN, EVENANDODD, gen_pt)
   for s in sites:
      dest[s] -= s.sign * 0.5 * gen_pt[s]

   gather(src, TUP, EVENANDODD, gen_pt)
   for s in sites:
      dest[s] += 0.5 * gen_pt[s]

   gather(src, TDN, EVENANDODD, gen_pt)
   for s in sites:
      dest[s] -= 0.5 * gen_pt[s]

   for s in sites:
      dest[s] = isign * dest[s] + s.phi * src[s]
